In [1]:
import pandas as pd

# Baca file CSV
df = pd.read_csv('cnbc_fix_dataset.csv')

# Ubah nilai -1 menjadi 0 di kolom 'polarity'
df['polarity'] = df['polarity'].replace(-1, 0)

# Tampilkan beberapa baris pertama untuk memverifikasi perubahan
print(df['polarity'].head())

# Opsional: Simpan kembali ke file CSV
df.to_csv('cnbc.csv', index=False)

0    0
1    0
2    0
3    0
4    1
Name: polarity, dtype: int64


In [14]:
df.head(5)

,Content,Token,Stop_article,Stem_article,Prepos_article,polarity_score,polarity
0,pemerintah tengah bersiap merealisasikan ekspo...,"['pemerintah', 'tengah', 'bersiap', 'merealisa...","['pemerintah', 'merealisasikan', 'ekspor', 'pa...",perintah realisasi ekspor pasir laut presiden ...,"['perintah', 'realisasi', 'ekspor', 'pasir', '...",-12,0
1,direktur jenderal bea cukai kementerian keuang...,"['direktur', 'jenderal', 'bea', 'cukai', 'keme...","['direktur', 'jenderal', 'bea', 'cukai', 'keme...",direktur jenderal bea cukai menteri uang askol...,"['direktur', 'jenderal', 'bea', 'cukai', 'ment...",-39,0
2,ketua umum ikatan pengusaha konveksi berkarya ...,"['jakarta', 'cnbc', 'indonesia', 'ketua', 'umu...","['jakarta', 'cnbc', 'indonesia', 'ketua', 'ika...",ketua ikat usaha konveksi karya ipkb nandi her...,"['ketua', 'ikat', 'usaha', 'konveksi', 'karya'...",-58,0
3,pengusaha tekstil di dalam negeri menyoroti pe...,"['pengusaha', 'tekstil', 'di', 'dalam', 'neger...","['pengusaha', 'tekstil', 'negeri', 'menyoroti'...",usaha tekstil negeri sorot nyata menteri uang ...,"['usaha', 'tekstil', 'negeri', 'sorot', 'nyata...",-85,0
4,presiden joko widodo jokowi memanggil beberapa...,"['presiden', 'joko', 'widodo', 'jokowi', 'mema...","['presiden', 'joko', 'widodo', 'jokowi', 'mema...",presiden joko widodo jokowi panggil menteri ra...,"['presiden', 'joko', 'widodo', 'jokowi', 'pang...",15,1


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from wordcloud import WordCloud
import re

import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.losses import categorical_crossentropy
from keras.optimizers import Adam

[nltk_data] Downloading package punkt to C:\Users\Anjelita
[nltk_data]     Malik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Tokenizing and padding sequences
max_features = 2000  # Adjust as necessary
max_length = 300

In [5]:
# Initialize Tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Stem_article'].values)

# Convert texts to sequences
X = tokenizer.texts_to_sequences(df['Stem_article'].values)
X = pad_sequences(X, padding='post', maxlen=max_length)

# Convert labels to a format acceptable by Keras
Y = pd.get_dummies(df['polarity'])
labels = Y.columns
Y = Y.values

In [6]:
# Display some samples of tokenization results
print("Contoh hasil tokenisasi dan padding:")
for i in range(3):
    print(X[i])

Contoh hasil tokenisasi dan padding:
[  15  235   10  231   80  375  402   96   18   18   15  548   68  181
   22  231   18  535 1164  147  231  482  118   32  529 1022  712  174
  254    5   23 1031 1080    5  231  555 1301   35  102   30    3    2
    5    6  251 1070  452  518   16    4  678  658  477  954  529   71
  174   87  125  153    5   23 1031 1080  836  231   10   69  290  869
  252  147  407   96  678   76   27  535  571  568    5   10   20  252
   22  168   26  932 1670   98  168  870  932  258  645  369 1607  285
  932  100 1671  390  932  754  390  820   20  933  369  955  390  249
  369  407   18  663   15 1155   18 1412  891 1608  777   95   46  231
  231  965  121   19  188  975  777  145  184  206 1609  249 1610    5
  231  555 1301  188  231   19  464 1637    4  976 1032  976 1032 1637
 1965  249  976 1032  177 1056 1194   10  231  348 1314  231    4  249
  947  249  489  231  194  175 1920  927  490  548   96  257  217   53
   10  231  235   15  535   37  562   32

In [7]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [8]:
embed_dim = 150

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(256, dropout=0.2, return_sequences=True)))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(Y_train.shape[1], activation='softmax'))

c:\Users\Anjelita Malik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Force the model to build by training on a small batch of real data
model.fit(X_train[:1], Y_train[:1], epochs=1, batch_size=1, verbose=1)

# Summarize the model
model.summary()

1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.0000e+00 - loss: 0.7015


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (1, 300, 150)          │       300,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (1, 300, 512)          │       833,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (1, 128)               │       328,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (1, 128)               │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (1, 128)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, 64)                │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (1, 2)                 │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,459,880 (17.01 MB)

 Trainable params: 1,486,626 (5.67 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,973,254 (11.34 MB)

In [10]:
batch_size = 32
epochs = 100
# Membuat callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metrik yang dipantau (bisa juga 'val_accuracy')
    patience=3,          # Jumlah epoch untuk menunggu sebelum berhenti jika tidak ada perbaikan
    min_delta=0.001,     # Perubahan minimum yang dianggap sebagai perbaikan
    mode='min',          # 'min' untuk loss, 'max' untuk accuracy
    restore_best_weights=True  # Mengembalikan bobot terbaik yang ditemukan selama pelatihan
)

# Menambahkan callback ke model.fit
history = model.fit(
    X_train, Y_train,
    epochs=epochs,  # Jumlah maksimum epoch
    batch_size=batch_size,
    validation_data=(X_test, Y_test),
    callbacks=[early_stopping],  # Menambahkan callback
    verbose=1
)

Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.6709 - loss: 0.6386 - val_accuracy: 0.7675 - val_loss: 0.5503
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.7045 - loss: 0.6130 - val_accuracy: 0.7591 - val_loss: 0.5547
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.7365 - loss: 0.5835 - val_accuracy: 0.7675 - val_loss: 0.5650
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.7209 - loss: 0.5968 - val_accuracy: 0.7647 - val_loss: 0.6030


In [11]:
pred = model.predict(X_test)

print(classification_report(Y_test.argmax(axis=1), pred.argmax(axis=1)))

12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 525ms/step
              precision    recall  f1-score   support

           0       0.77      1.00      0.87       274
           1       0.00      0.00      0.00        83

    accuracy                           0.77       357
   macro avg       0.38      0.50      0.43       357
weighted avg       0.59      0.77      0.67       357



c:\Users\Anjelita Malik\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anjelita Malik\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Anjelita Malik\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh